In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import numpy as np
from datetime import date

executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=True)

# URL of page to be scraped
url = 'http://www.licigob.com.mx/cgi-bin/cupisa_optodhue93hefg8yu4iewfrgh.asp'

browser.visit(url)
time.sleep(5)
response = browser.html
soup = bs(response, 'html.parser')

results = soup.find_all('table')
allrows = results[3].find_all("tr")
test_rows = allrows[1:]
licitaciones = []
row_counter = 0
concurso_dict = {}
for rows in test_rows:
    data = rows.find_all("td")
    firsttext = data[0].text
    if firsttext == "Dependencia":
        pass
    elif firsttext == "Bases":
        link = rows.find("a")["href"]
        concurso_dict["link"] = link
    elif len(data) > 3:
        concurso_dict["interno"] = "CUPISA"
        concurso_dict["Dependencia"] = data[0].text
        concurso_dict["Lugar"] = data[1].text
        if data[2].text.strip() == "":
            fechaactual = date.today().strftime("%d/%m/%Y")
            concurso_dict["fecha_de_publicacion"] = fechaactual
        else:
            concurso_dict["fecha_de_publicacion"] = data[2].text.strip()
        concurso_dict["fecha_de_presentacion"] = data[3].text
        concurso_dict["Modalidad"] = data[4].text
        concurso_dict["Numero_de_procedimiento"] = data[5].text
    elif len(data) == 1:
        concurso_dict["Descripcion"] = data[0].text
        licitaciones.append(concurso_dict)
        concurso_dict = {}


url = 'http://www.licigob.com.mx/cgi-bin/infineo_todfh298309qewtd672fgy354wte.asp'

browser.visit(url)
time.sleep(5)
response = browser.html
soup = bs(response, 'html.parser')

results = soup.find_all('table')
allrows = results[3].find_all("tr")
test_rows = allrows[1:]
row_counter = 0
concurso_dict = {}
for rows in test_rows:
    data = rows.find_all("td")
    firsttext = data[0].text
    if firsttext == "Dependencia":
        pass
    elif firsttext == "Bases":
        link = rows.find("a")["href"]
        concurso_dict["link"] = link
    elif len(data) > 3:
        concurso_dict["interno"] = "URALES"
        concurso_dict["Dependencia"] = data[0].text
        concurso_dict["Lugar"] = data[1].text
        if data[2].text.strip() == "":
            fechaactual = date.today().strftime("%d/%m/%Y")
            concurso_dict["fecha_de_publicacion"] = fechaactual
        else:
            concurso_dict["fecha_de_publicacion"] = data[2].text.strip()
        concurso_dict["fecha_de_presentacion"] = data[3].text
        concurso_dict["Modalidad"] = data[4].text
        concurso_dict["Numero_de_procedimiento"] = data[5].text
    elif len(data) == 1:
        concurso_dict["Descripcion"] = data[0].text
        licitaciones.append(concurso_dict)
        concurso_dict = {}





In [39]:
df = pd.DataFrame(licitaciones)
df["unique"] = df["Numero_de_procedimiento"].duplicated()
df.loc[df["unique"] == True, "interno"] = "COMBINADO"
df.drop("unique", axis=1, inplace=True)
df_nodup=df.drop_duplicates("Numero_de_procedimiento", keep="last")
df_nodup["fecha_de_publicacion"] = pd.to_datetime(df_nodup["fecha_de_publicacion"], dayfirst=True,  format="%d/%m/%Y")
df_nodup["fecha_de_presentacion"] = pd.to_datetime(df_nodup["fecha_de_presentacion"], dayfirst=True,  format="%d/%m/%Y")
df_nodup.sort_values(by=["fecha_de_publicacion"], ascending=False ,inplace=True)

In [43]:
df_nodup["Nuevo"] = "No"

In [44]:
from datetime import datetime, timedelta
df_nodup.loc[df_nodup["fecha_de_publicacion"] >= pd.to_datetime(date.today() - timedelta(days=2)), "Nuevo"] = "Si"

,interno,Dependencia,Lugar,fecha_de_publicacion,fecha_de_presentacion,Modalidad,Numero_de_procedimiento,link,Descripcion,Nuevo
0,CUPISA,"Municipio de Galeana, Nuevo León",Nuevo León,2020-06-15,2020-07-02,Licitación Pública,PMG-FSM-02/2017,/cgi-bin/pdf/nodisponible.html,suministro e instalación de luminarias led,Si
272,URALES,VER-Comisión del Agua del Estado de Veracruz-S...,Veracruz,2020-06-15,2020-06-25,04. Invitación a Cuando Menos Tres Personas LO...,2121718,https://compranet.hacienda.gob.mx/esop/toolkit...,CONSTRUCCIÓN DE LA PLANTA DE TRATAMIENTO DE AG...,Si
281,URALES,SLP-Villa de Ramos-MUNICIPIO DE VILLA DE RAMOS...,San Luis Potosí,2020-06-15,2020-06-22,02. Licitación Pública LOPSRM,2120558,https://compranet.hacienda.gob.mx/esop/toolkit...,?Construcción De Alcantarillado Sanitario 3ª. ...,Si
280,URALES,SON-Secretaria de Agricultura Ganadería Recurs...,Sonora,2020-06-15,2020-06-30,02. Licitación Pública LOPSRM,2121259,https://compranet.hacienda.gob.mx/esop/toolkit...,AMPLIACIÓN DE 3160 ML DE RED DE AGUA POTABLE C...,Si
279,URALES,SON-Secretaria de Agricultura Ganadería Recurs...,Sonora,2020-06-15,2020-06-30,02. Licitación Pública LOPSRM,2121302,https://compranet.hacienda.gob.mx/esop/toolkit...,EQUIPAMIENTO DE CÁRCAMO DE BOMBEO CONSTRUCCIÓN...,Si
278,URALES,TAB-Emiliano Zapata-Aytto. de Emiliano Zapata ...,Tabasco,2020-06-15,2020-06-24,04. Invitación a Cuando Menos Tres Personas LO...,2119878,https://compranet.hacienda.gob.mx/esop/toolkit...,I0031.- CONSTRUCCIÓN DE DRENAJE SANITARIO EN L...,Si
277,URALES,SLP-Santo Domingo-MUNICIPIO DE SANTO DOMINGO 8...,San Luis Potosí,2020-06-15,2020-06-23,02. Licitación Pública LOPSRM,2120579,https://compranet.hacienda.gob.mx/esop/toolkit...,Construcción De Red De Drenaje Y Colector Sani...,Si
276,URALES,SLP-Villa de Arriaga-H. AYUNTAMIENTO DE VILLA ...,San Luis Potosí,2020-06-15,2020-06-24,02. Licitación Pública LOPSRM,2120728,https://compranet.hacienda.gob.mx/esop/toolkit...,Construcción De Red De Drenaje Y Colector Sani...,Si
275,URALES,OAX-Santo Domingo Tehuantepec-Regiduría de hac...,Oaxaca,2020-06-15,2020-06-26,04. Invitación a Cuando Menos Tres Personas LO...,2121271,https://compranet.hacienda.gob.mx/esop/toolkit...,ESTUDIO DIAGNOSTICO Y PROYECTO EJECUTIVO DEL S...,Si
274,URALES,SON-Secretaria de Agricultura Ganadería Recurs...,Sonora,2020-06-15,2020-06-30,02. Licitación Pública LOPSRM,2121322,https://compranet.hacienda.gob.mx/esop/toolkit...,ELABORACIÓN DE PROYECTO EJECUTIVO INTERCOMUNIT...,Si


In [36]:
antier= date.today() - timedelta(days=2)
pd.to_datetime(antier)

Timestamp('2020-06-14 00:00:00')